In [0]:
%sql
USE initialcatalog.initialschema;
DESCRIBE SCHEMA EXTENDED initialcatalog.initialschema

In [0]:
#Files available in the volume
spark.sql(f"LIST'/Volumes/initialcatalog/initialschema/initialvolume'").display()

spark.sql(f'''
          select * from csv.`/Volumes/initialcatalog/initialschema/initialvolume`''').display()


In [0]:
from pyspark.sql.types import StructType, StructField, StringType,DecimalType
# Read CSV files
schema = StructType([ StructField("2017 NAICS Code", StringType(), True),
                     StructField("2017 NAICS Title", StringType(), True),
                     StructField("GHG", StringType(), True),
                     StructField("Unit", StringType(), True),
                     StructField("Supply Chain Emission Factors without Margins", DecimalType(10,3), True),
                     StructField("Margins of Supply Chain Emission Factors", DecimalType(10,3), True),
                     StructField("Supply Chain Emission Factors with Margins", DecimalType(10,3), True),
                     StructField("Reference USEEIO Code", StringType(), True),
                     
                     
                   ])

df = spark.read.format("csv").schema(schema).option("header", "true").load("/Volumes/initialcatalog/initialschema/initialvolume/SupplyChainGHGEmissionFactors_v1.2_NAICS_CO2e_USD2021.csv")

df = df.withColumnRenamed("2017 NAICS Code","NAICS_Code")\
.withColumnRenamed("2017 NAICS Title","NAICS_Title")\
.withColumnRenamed("Supply Chain Emission Factors without Margins","SUPLY_CHN_EMISN_FACT_WITHOUT_MARGINS")\
.withColumnRenamed("Margins of Supply Chain Emission Factors","MARGINS_OF_SUPLY_CHN_EMISN_FACT")\
.withColumnRenamed("Supply Chain Emission Factors with Margins","SUPLY_CHN_EMISN_FACT_WITH_MARGINS")\
.withColumnRenamed("Reference USEEIO Code","USEEIO_CODE")

df.write.mode("overwrite").option("overrieschema","true").saveAsTable("initialcatalog.initialschema.tbl_supply_chain_emission_factors")

In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.default.tbl_supply_chain_emission_factors")
spark.sql("select * from tbl_supply_chain_emission_factors").display()

In [0]:
df = spark.read.table("initialcatalog.initialschema.tbl_supply_chain_emission_factors")
df.display()

In [0]:
#update a table 
from delta.tables import DeltaTable

deltatable = DeltaTable.forName(spark, "initialcatalog.initialschema.tbl_supply_chain_emission_factors")
deltatable.update(
    condition="NAICS_Code = '111120'",
    set={"NAICS_Title": "'Manufacturing'"}
)

In [0]:
df = spark.read.table("initialcatalog.initialschema.tbl_supply_chain_emission_factors")
df = df.where(df.NAICS_Code == '111120')
df.display()

In [0]:
%sql
--time travel

DESCRIBE HISTORY initialcatalog.initialschema.tbl_supply_chain_emission_factors

In [0]:
df = spark.read.table("initialcatalog.initialschema.tbl_supply_chain_emission_factors@v0")
df = df.where(df.NAICS_Code == '111120')
df.display()

In [0]:
%sql
select * from initialcatalog.initialschema.tbl_supply_chain_emission_factors version as of 0
where NAICS_Code = '111120'
    
